In [ ]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import pandas as pd
import pdo_functions
import importlib
importlib.reload(pdo_functions)

In [ ]:
# Open datasets: want CAPE, S06, NDSEV, 6 km wind and surface wind
cape = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/wrf_cape/era5_daily_max_mucape_*s.nc', combine='nested', concat_dim='year')['__xarray_dataarray_variable__']
s06 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/era5_s06_*s.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']

u10 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/u10/*s/e5.oper.an.sfc.128_165_10u.*.nc', combine='nested', concat_dim='time')['VAR_10U']
v10 = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/v10/*s/e5.oper.an.sfc.128_166_10v.*.nc', combine='nested', concat_dim='time')['VAR_10V']

u6k = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/u_6k/*.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']
v6k = xr.open_mfdataset('/hurrell-scratch2/ivyglade/pdo/ERA5_S06/v_6k/*.nc', combine='nested', concat_dim='time')['__xarray_dataarray_variable__']

In [ ]:
# subset only MAM
cape_mam = cape.sel(date=cape.date.where(
                         cape.date.str.startswith('03-') |
                         cape.date.str.startswith('04-') |
                         cape.date.str.startswith('05-'),
                         drop=True)).load()

In [ ]:
# flatten to have a single time dimension
cape_mam_stack = cape_mam.stack(time=('year', 'date'))

# Convert time to datetime
year = cape_mam_stack['time'].get_index('time').get_level_values('year')
date = cape_mam_stack['time'].get_index('time').get_level_values('date')

datetime = pd.to_datetime(year.astype(str) + '-' + date.astype(str))

cape_mam_stack = cape_mam_stack.assign_coords(time=datetime)

In [ ]:
# Subsetting the S06 dataset
# Conus
s06_conus = s06.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

# MAM only
s06_conus_mam = s06_conus.sel(time=s06_conus.time.dt.month.isin([3, 4, 5])).transpose('latitude', 'longitude', 'time').load()

In [ ]:
# Subsetting wind data
# CONUS
u10_conus = u10.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
v10_conus = v10.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
u6k_conus = u6k.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))
v6k_conus = v6k.sel(latitude=slice(49, 24)).sel(longitude=slice(235, 294))

# MAM only
u10_conus_mam = u10_conus.sel(time=u10_conus.time.dt.month.isin([3, 4, 5])).load()
v10_conus_mam = v10_conus.sel(time=v10_conus.time.dt.month.isin([3, 4, 5])).load()
u6k_conus_mam = u6k_conus.sel(time=u6k_conus.time.dt.month.isin([3, 4, 5])).load()
v6k_conus_mam = v6k_conus.sel(time=v6k_conus.time.dt.month.isin([3, 4, 5])).load()

In [ ]:
# Compute wind speed
wind_sfc = np.sqrt(u10_conus_mam**2 + v10_conus_mam**2)
wind_6k = np.sqrt(u6k_conus_mam**2 + v6k_conus_mam**2)

In [ ]:
# Align time dimensions
cape_mam_stack['time'] = s06_conus_mam['time']

In [ ]:
# CAPES06
capes06 = cape_mam_stack * s06_conus_mam

In [ ]:
np.shape(capes06)

In [ ]:
np.shape(s06_conus_mam)

In [ ]:
np.shape(wind_sfc.transpose('latitude', 'longitude', 'time'))

In [ ]:
np.shape(wind_6k.transpose('latitude', 'longitude', 'time'))

In [ ]:
np.shape(cape_mam_stack)

In [ ]:
ndsev = xr.where((capes06 > 10000) & (s06_conus_mam > 5) & (wind_sfc.transpose('latitude', 'longitude', 'time') > 5) & (wind_6k.transpose('latitude', 'longitude', 'time') > 5) & (wind_6k.transpose('latitude', 'longitude', 'time') > wind_sfc.transpose('latitude', 'longitude', 'time')) & (cape_mam_stack > 100), 1, 0)

In [ ]:
ndsev

In [ ]:
ndsev.to_netcdf('/hurrell-scratch2/ivyglade/pdo/ndsev/ndsev_1940-2024_mam_corrected.nc')

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-97, central_latitude=-36.5)))

bounds = [0, 1, 2, 3, 4, 6, 8, 10, 12, 16, 20, 24, 28]

norm = c.BoundaryNorm(bounds, plt.get_cmap('Reds').N)

ax.add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')

ax.pcolormesh(ndsev['longitude'], ndsev['latitude'], ndsev.groupby('time.year').sum().mean(dim='year'), transform=ccrs.PlateCarree(), cmap='Reds', norm=norm)